In [1]:
import pandas as pd

In [2]:
# read stress.csv
df = pd.read_csv('Stress.csv')

In [9]:
df.head(5)

subreddit post_id sentence_range  \
0              ptsd  8601tu       (15, 20)   
1        assistance  8lbrx9         (0, 5)   
2              ptsd  9ch1zh       (15, 20)   
3     relationships  7rorpp        [5, 10]   
4  survivorsofabuse  9p2gbc         [0, 5]   

                                                text  label  confidence  \
0  He said he had not felt that way before, sugge...      1         0.8   
1  Hey there r/assistance, Not sure if this is th...      0         1.0   
2  My mom then hit me with the newspaper and it s...      1         0.8   
3  until i met my new boyfriend, he is amazing, h...      1         0.6   
4  October is Domestic Violence Awareness Month a...      1         0.8   

   social_timestamp  
0        1521614353  
1        1527009817  
2        1535935605  
3        1516429555  
4        1539809005

In [16]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("hsaglamlar/autotrain-stress_v2-1178743973")

tokenizer = AutoTokenizer.from_pretrained("hsaglamlar/autotrain-stress_v2-1178743973")

inputs = tokenizer("I love AutoTrain", return_tensors="pt")

outputs = model(**inputs)

In [17]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.1542, -1.3433]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [12]:
inputs = tokenizer(df.iloc[0]['text'], return_tensors="pt")

outputs = model(**inputs)
print(outputs.logits)

tensor([[ 0.6629, -0.7704]], grad_fn=<AddmmBackward0>)


In [13]:
import torch
import torch.nn.functional as F

def clasificar_estres(logits):
    # Aplicar la función softmax a los logits
    probabilities = F.softmax(logits, dim=1)
    
    # Obtener el índice de la clase con mayor probabilidad
    predicted_class = torch.argmax(probabilities, dim=1).item()
    
    # Determinar la clasificación basada en el índice predicho
    if predicted_class == 0:  # Si el índice predicho es 0 (no estrés)
        return 0
    else:  # Si el índice predicho es 1 (estrés)
        return 1


In [14]:
clasificar_estres(outputs.logits)

0

In [19]:
stress_pred = []
for i in range(len(df)):
    inputs = tokenizer(df.iloc[i]['text'], return_tensors="pt")
    outputs = model(**inputs)
    stress_pred.append(clasificar_estres(outputs.logits))

In [18]:
import numpy as np

In [22]:
stress_pred = np.array(stress_pred)

In [25]:
y = df['label'].to_numpy()

In [26]:
mse = ((stress_pred - y)**2).mean()

In [28]:
accuracy = (stress_pred == y).mean()

In [29]:
accuracy

0.5708245243128964

In [31]:
print(df.iloc[0]['text'])

He said he had not felt that way before, suggeted I go rest and so ..TRIGGER AHEAD IF YOUI'RE A HYPOCONDRIAC LIKE ME: i decide to look up "feelings of doom" in hopes of maybe getting sucked into some rabbit hole of ludicrous conspiracy, a stupid "are you psychic" test or new age b.s., something I could even laugh at down the road. No, I ended up reading that this sense of doom can be indicative of various health ailments; one of which I am prone to.. So on top of my "doom" to my gloom..I am now f'n worried about my heart. I do happen to have a physical in 48 hours.


In [54]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("ValenHumano/roberta-base-bne-detector-de-stress")

tokenizer = AutoTokenizer.from_pretrained("ValenHumano/roberta-base-bne-detector-de-stress")

inputs = tokenizer("I love AutoTrain", return_tensors="pt")

outputs = model(**inputs)

In [44]:
stress_pred = []
for i in range(len(df)):
    inputs = tokenizer(df.iloc[i]['text'], return_tensors="pt", max_length=512, truncation=True)
    # check if the input is too long for the model
    outputs = model(**inputs)
    stress_pred.append(clasificar_estres(outputs.logits))
stress_pred = np.array(stress_pred)
mse = ((stress_pred - y)**2).mean()
accuracy = (stress_pred == y).mean()
print('mse:', mse, '\naccuracy: ', accuracy)

mse: 0.23291050035236083 
accuracy:  0.7670894996476392


In [16]:
# open txt file
with open(r'C:\Users\Pc\Documents\Talk_to_Eve_mvp\documents\conversations_examples\conversations.txt', 'r') as file:
    text = file.read()

text = text.split('\n')
# delete empty lines
text = [x for x in text if x]
folders_idx = [i for i, x in enumerate(text) if x.startswith('# Folder')]

In [19]:
pages = np.array([folders_idx[i+1] - folders_idx[i] for i in range(len(folders_idx)-1)])
id_max, max_num =  np.argmax(pages), np.max(pages)
print(id_max, max_num)

257 33


In [20]:
conversation = text[folders_idx[257]:folders_idx[258]][1:]

In [21]:
conversation2 = text[folders_idx[3]:folders_idx[4]][1:]

In [55]:
stress_pred = []
stress_proba = []
for row in conversation:
    if row.startswith('User:'):
        inputs = tokenizer(row[6:], return_tensors="pt", max_length=512, truncation=True)
        outputs = model(**inputs)
        stress_pred.append(clasificar_estres(outputs.logits))
        stress_proba.append(F.softmax(outputs.logits, dim=1).tolist()[0][1])
        

In [62]:
outputs.logits.softmax(dim=1).detach().numpy()[0][1]

0.26596072

In [56]:
stress_pred

[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0]

In [61]:
np.array(stress_pred).mean()

0.625

In [60]:
np.round(np.array(stress_proba),2).mean()

0.6368750000000001

In [ ]:
conversation

In [63]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("ShreyaR/finetuned-roberta-depression")

tokenizer = AutoTokenizer.from_pretrained("ShreyaR/finetuned-roberta-depression")

inputs = tokenizer("I love AutoTrain", return_tensors="pt")

outputs = model(**inputs)

config.json: 100%|██████████| 735/735 [00:00<00:00, 724kB/s]
pytorch_model.bin: 100%|██████████| 499M/499M [02:10<00:00, 3.81MB/s] 
tokenizer_config.json: 100%|██████████| 349/349 [00:00<?, ?B/s] 
vocab.json: 100%|██████████| 798k/798k [00:00<00:00, 1.65MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 1.29MB/s]
tokenizer.json: 100%|██████████| 2.11M/2.11M [00:00<00:00, 3.35MB/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<?, ?B/s] 


In [67]:
depression_pred = []
depression_proba = []
for row in conversation:
    if row.startswith('User:'):
        inputs = tokenizer(row[6:], return_tensors="pt")
        outputs = model(**inputs)
        depression_pred.append(clasificar_estres(outputs.logits))
        depression_proba.append(outputs.logits.softmax(dim=1).detach().numpy()[0][1])
        

In [68]:
depression_pred

[0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [71]:
np.round(np.array(depression_proba), 3)

array([0.003, 0.997, 0.998, 0.953, 0.995, 0.998, 0.011, 0.026, 0.026,
       0.003, 0.002, 0.002, 0.978, 0.04 , 0.005, 0.152], dtype=float32)

In [91]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import BertTokenizer

model = AutoModelForSequenceClassification.from_pretrained("Phoshco/AnxietyvsN")

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

inputs = tokenizer("I love AutoTrain", return_tensors="pt")

outputs = model(**inputs)

In [92]:
anx_pred = []
anx_proba = []
for row in conversation2:
    if row.startswith('User:'):
        inputs = tokenizer(row[6:], return_tensors="pt")
        outputs = model(**inputs)
        anx_pred.append(clasificar_estres(outputs.logits))
        anx_proba.append(outputs.logits.softmax(dim=1).detach().numpy()[0][1])

In [93]:
anx_pred

[1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0]

In [94]:
anx_proba

[0.9924154,
 0.9972959,
 0.9920476,
 0.9993698,
 0.7504077,
 0.002009445,
 0.9988257,
 0.999032,
 0.95685226,
 0.0043498124,
 0.9970674,
 0.907122,
 0.13471437]

In [14]:
conversation2

NameError: name 'conversation2' is not defined

In [46]:
from transformers import pipeline

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

sentences = ["I am not having a great day"]

model_outputs = classifier(conversation)

In [3]:
import pandas as pd

In [51]:
df = pd.DataFrame(model_outputs[0]).T
# change column to the first row
df.columns = df.iloc[0]
df = df[1:]
# sort columns by name
df = df.sort_index(axis=1)
df

label admiration amusement     anger annoyance approval    caring confusion  \
score   0.002623  0.001739  0.006845  0.066436  0.02274  0.009462  0.270765   

label curiosity   desire disappointment  ...      love nervousness   neutral  \
score  0.010662  0.00304       0.309621  ...  0.004324    0.087151  0.056518   

label  optimism     pride realization    relief   remorse  sadness  surprise  
score  0.004918  0.000936    0.045106  0.005228  0.020331  0.18996  0.003037  

[1 rows x 28 columns]

In [59]:

for i, chat in enumerate(conversation):
    if i==0:
        df = pd.DataFrame(classifier(chat)[0]).T
        # change column to the first row
        df.columns = df.iloc[0]
        df = df[1:]
        # sort columns by name
        df = df.sort_index(axis=1)
    else:
        ap = pd.DataFrame(classifier(chat)[0]).T
        # change column to the first row
        ap.columns = ap.iloc[0]
        ap = ap[1:]
        # sort columns by name
        ap = ap.sort_index(axis=1)
        df.loc[i] = ap.values[0]
df.reset_index(drop=True, inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True)
df.mean(axis=0).sort_values(ascending=False)

In [66]:
df.head(2)

label admiration amusement     anger annoyance approval    caring confusion  \
0       0.002623  0.001739  0.006845  0.066436  0.02274  0.009462  0.270765   
1       0.004544  0.006381  0.005776  0.007354  0.01561  0.047061   0.06084   

label curiosity    desire disappointment  ...      love nervousness   neutral  \
0      0.010662   0.00304       0.309621  ...  0.004324    0.087151  0.056518   
1      0.390826  0.008238       0.023125  ...  0.012225    0.008611  0.040987   

label  optimism     pride realization    relief   remorse   sadness  surprise  
0      0.004918  0.000936    0.045106  0.005228  0.020331   0.18996  0.003037  
1      0.007357  0.000381    0.008307   0.00203   0.69425  0.294414  0.003145  

[2 rows x 28 columns]

In [ ]:
ee = np.ones(32)
bb = np.zeros(32)
dff = pd.DataFrame([ee, bb]).T
dff

In [71]:
# combine dff and df
rr = pd.concat([dff, df], axis=1)
rr.head(2)

0    1 admiration amusement     anger annoyance approval    caring  \
0  1.0  0.0   0.002623  0.001739  0.006845  0.066436  0.02274  0.009462   
1  1.0  0.0   0.004544  0.006381  0.005776  0.007354  0.01561  0.047061   

  confusion curiosity  ...      love nervousness   neutral  optimism  \
0  0.270765  0.010662  ...  0.004324    0.087151  0.056518  0.004918   
1   0.06084  0.390826  ...  0.012225    0.008611  0.040987  0.007357   

      pride realization    relief   remorse   sadness  surprise  
0  0.000936    0.045106  0.005228  0.020331   0.18996  0.003037  
1  0.000381    0.008307   0.00203   0.69425  0.294414  0.003145  

[2 rows x 30 columns]

In [77]:
def intercalar_con_nan(df, last_role):
    df_intercalado = pd.DataFrame(index=np.arange(len(df)*2 - 1 if last_role == 'Adam' else len(df)*2), columns=df.columns)

    for column in df.columns:
        output = np.empty(len(df)*2 - 1 if last_role == 'Adam' else len(df)*2)
        output[: : 2] = df[column].values
        output[1: : 2] = np.nan
        df_intercalado[column] = output
    
    return df_intercalado

In [80]:
df.columns

Index(['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
       'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
       'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
       'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride',
       'realization', 'relief', 'remorse', 'sadness', 'surprise'],
      dtype='object', name='label')

In [81]:
dfi = intercalar_con_nan(df, 'Eve')
dfi

label  admiration  amusement     anger  annoyance  approval    caring  \
0        0.002623   0.001739  0.006845   0.066436  0.022740  0.009462   
1             NaN        NaN       NaN        NaN       NaN       NaN   
2        0.004544   0.006381  0.005776   0.007354  0.015610  0.047061   
3             NaN        NaN       NaN        NaN       NaN       NaN   
4        0.001564   0.003469  0.029576   0.304984  0.019755  0.002411   
..            ...        ...       ...        ...       ...       ...   
59            NaN        NaN       NaN        NaN       NaN       NaN   
60       0.003536   0.001540  0.000732   0.004602  0.543609  0.002612   
61            NaN        NaN       NaN        NaN       NaN       NaN   
62       0.859860   0.001320  0.001293   0.002037  0.170998  0.030473   
63            NaN        NaN       NaN        NaN       NaN       NaN   

label  confusion  curiosity    desire  disappointment  ...      love  \
0       0.270765   0.010662  0.003040        0.309621  ...  0.004324   
1            NaN        NaN       NaN             NaN  ...       NaN   
2       0.060840   0.390826  0.008238        0.023125  ...  0.012225   
3            NaN        NaN       NaN             NaN  ...       NaN   
4       0.053873   0.003040  0.001361        0.049652  ...  0.000556   
..           ...        ...       ...             ...  ...       ...   
59           NaN        NaN       NaN             NaN  ...       NaN   
60      0.002374   0.001667  0.001688        0.001217  ...  0.001012   
61           NaN        NaN       NaN             NaN  ...       NaN   
62      0.001009   0.002089  0.009442        0.001046  ...  0.084387   
63           NaN        NaN       NaN             NaN  ...       NaN   

label  nervousness   neutral  optimism     pride  realization    relief  \
0         0.087151  0.056518  0.004918  0.000936     0.045106  0.005228   
1              NaN       NaN       NaN       NaN          NaN       NaN   
2         0.008611  0.040987  0.007357  0.000381     0.008307  0.002030   
3              NaN       NaN       NaN       NaN          NaN       NaN   
4         0.001270  0.141014  0.004513  0.000450     0.020457  0.001014   
..             ...       ...       ...       ...          ...       ...   
59             NaN       NaN       NaN       NaN          NaN       NaN   
60        0.000198  0.522041  0.019715  0.000345     0.014994  0.000815   
61             NaN       NaN       NaN       NaN          NaN       NaN   
62        0.000403  0.022310  0.049980  0.010247     0.004396  0.003392   
63             NaN       NaN       NaN       NaN          NaN       NaN   

label   remorse   sadness  surprise  
0      0.020331  0.189960  0.003037  
1           NaN       NaN       NaN  
2      0.694250  0.294414  0.003145  
3           NaN       NaN       NaN  
4      0.000987  0.003045  0.001807  
..          ...       ...       ...  
59          NaN       NaN       NaN  
60     0.000215  0.000554  0.000480  
61          NaN       NaN       NaN  
62     0.000558  0.001141  0.001378  
63          NaN       NaN       NaN  

[64 rows x 28 columns]

In [83]:
df.mean(axis=0)

label
admiration        0.081236
amusement         0.002183
anger             0.003529
annoyance         0.024454
approval          0.154845
caring            0.045323
confusion         0.180421
curiosity         0.144654
desire            0.011341
disappointment    0.038985
disapproval       0.058098
disgust           0.001661
embarrassment     0.002433
excitement        0.005358
fear              0.003075
gratitude         0.035849
grief             0.001358
joy               0.019429
love              0.023552
nervousness       0.007348
neutral           0.277251
optimism           0.07706
pride             0.001301
realization       0.019211
relief            0.002823
remorse           0.025329
sadness           0.038199
surprise          0.002387
dtype: object

In [86]:
dfi.mean()

label
admiration        0.081236
amusement         0.002183
anger             0.003529
annoyance         0.024454
approval          0.154845
caring            0.045323
confusion         0.180421
curiosity         0.144654
desire            0.011341
disappointment    0.038985
disapproval       0.058098
disgust           0.001661
embarrassment     0.002433
excitement        0.005358
fear              0.003075
gratitude         0.035849
grief             0.001358
joy               0.019429
love              0.023552
nervousness       0.007348
neutral           0.277251
optimism          0.077060
pride             0.001301
realization       0.019211
relief            0.002823
remorse           0.025329
sadness           0.038199
surprise          0.002387
dtype: float64